# Libraries

In [334]:
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tensorflow import keras
import scienceplots
plt.style.use(['science', 'notebook'])
import time

import seaborn as sns

import minimal as mn
importlib.reload(mn) 


import keras
from keras import models
from keras import layers

modulo cargado...


# Synthetic images

In [354]:
class SyntheticImages:

    def __init__(self, number_configs: int, training: list[str]=['all'], L=40):
        if training == ['all']:
            training = ['para', 'ferro', 'neel', 'stripe']
        else:
            training = [element.lower() for element in training]

        self.number_configs = number_configs
        self.training = training
        self.L = L

    def spin_gen(self, conf: str):
        '''
        Generates a spin configuration for the given configuration. 
        The resulting data is a tuple of the different configurations each one can have.
        For example, ferromagnetic configurations can be spin up ferromagnetic or spin down ferromagnetic.
        In this case, the tuple will contain two LxL matrices, one per each type.
        '''
        spin_conf = []
        if conf == 'ferro':
            spin_conf = [np.ones((self.L, self.L)).astype(int), 
                        -np.ones((self.L, self.L)).astype(int)]
        elif conf == 'neel':
            spin = np.fromfunction(lambda i, j: (-1)**(i + j + (conf == 4)), 
                                (self.L, self.L)).astype(int)
            spin_conf = [spin, -spin]
        elif conf == 'stripe':
            spin = np.fromfunction(lambda i, j: (-1)**j, (self.L, self.L)).astype(int)
            spin_conf = [spin, -spin, spin.T, -spin.T]
        elif conf == 'para':
            spin_conf = np.random.choice([-1, 1], size=(self.L, self.L)).astype(int)
        return spin_conf

    def dataGenerator(self):
        ''' 
        Generates synthetic data given a number of configurations and the type of training we want to do.
        If 'all', then it will generate all possible configurations evenly distributed. There are 4 types of configurations.
        If the number of configurations is not divisible by 4, the remaining configurations will be generated in a paramagnetic manner.
        '''
        start_time = time.time()
        print("Generating synthetic data...")
        config_dict = {
            'para': 1,
            'ferro': 2,
            'neel': 2,
            'stripe': 4
        }

        labels_dict = {
            'para': 0,
            'ferro': 1,
            'neel': 2,
            'stripe': 3
        }

        selected_dict = {k: v for k, v in config_dict.items() if k in self.training}

        selected_labels = {k: v for k, v in labels_dict.items() if k in self.training}
        
        total_configs_per_selected = self.number_configs // len(selected_dict.values())
        remaining_configs = self.number_configs % len(selected_dict.values())

        train_images = []
        train_labels = []

        if 'para' in self.training:
            for _ in range(total_configs_per_selected):
                train_images.append(self.spin_gen('para'))
                train_labels.append(labels_dict['para'])
            del selected_dict['para'], selected_labels['para']
        
        for conf in selected_dict:
            total_conf = total_configs_per_selected
            extra_configs = total_conf % selected_dict[conf]    
                
            if extra_configs !=0:
                remaining_configs += extra_configs
            
            total_conf = total_conf // selected_dict[conf]

            for _ in range(total_conf):
                train_images.extend(self.spin_gen(conf))
                for _ in range(config_dict[conf]):
                    train_labels.append(labels_dict[conf])

        if 'para' in self.training:
            for _ in range(remaining_configs):
                train_images.append(self.spin_gen('para'))
                train_labels.append(labels_dict['para'])
        else:
            for _ in range(remaining_configs):
                train_images.append(self.spin_gen(self.training[0])[0])
                train_labels.append(labels_dict[self.training[0]])

        temp = list(zip(train_images, train_labels))
        random.shuffle(temp)
        train_images, train_labels = zip(*temp)

        print("Done!")

        end_time = time.time()
        elapsed_time = end_time - start_time
        print("Elapsed time:", elapsed_time, "seconds")
        return np.array(train_images), np.array(train_labels)


In [369]:
data = SyntheticImages(10000, training=['ferro','neel'], L=40)

In [370]:
train_images, train_labels = data.dataGenerator()

Generating synthetic data...
Done!
Elapsed time: 0.318206787109375 seconds


In [371]:
train_images.shape

(10000, 40, 40)

# Simulated images

# Neural networks